In [1]:
import logging
logger = logging.getLogger()
logger.setLevel(logging.INFO)

In [12]:
with open('../context-clustering/stopwords.txt') as f:
    stopwords = {line.strip().split()[0] for line in f if line.strip()}
print(list(stopwords)[:100])
word = 'горшок'
stopwords.add(word)
with open('../corpora/ad-nouns-contexts-100k/{}.txt'.format(word)) as f:
    texts = [[w for w in line.strip().split() if w not in stopwords] for line in f]
for text in texts[:3]:
    print(text)
print(len(texts))
#texts = texts[:1000]

['кто', 'вот', 'какой', 'если', 'кажется', 'два', 'впрочем', 'нас', 'вдруг', 'на', 'всех', 'ей', 'тебя', 'себе', 'свой', 'тем', 'между', 'по', 'сказала', 'даже', 'потом', 'тут', 'им', 'сейчас', 'разве', 'до', 'то', 'не', 'с', 'моя', 'мой', 'сегодня', 'опять', 'чтоб', 'из', '|', 'него', 'как', 'говорил', 'об', 'иногда', 'перед', 'ничего', 'эти', 'но', 'сказал', 'их', 'жизнь', 'про', 'и', 'ж', 'раз', 'наконец', 'себя', 'да', 'чуть', 'хорошо', 'бы', 'эту', 'мне', 'в', 'под', 'нибудь', 'чего', 'без', 'были', 'через', 'за', 'более', 'конечно', 'был', 'будет', 'сказать', 'где', 'почти', 'куда', 'сам', 'один', 'тот', 'еще', 'о', 'ни', 'или', 'так', 'мы', 'зачем', 'она', 'нее', 'тогда', 'никогда', 'больше', 'только', 'нельзя', 'он', 'надо', 'вас', 'вы', 'ему', 'всегда', 'здесь']
['час', 'время', 'инцидент', 'хватить', 'некоторый', 'легенда', 'говориться', 'золото', 'находиться', 'конец', 'радуга', 'глубина', '22', 'фут', 'почва', 'становиться']
['гнить', 'нераскрытый', 'тайна', 'коридор', 'боя

In [41]:
from gensim import corpora
dictionary = corpora.Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]
print(corpus[0], corpus[1])

[(0, 1), (1, 1), (2, 1), (3, 1), (4, 1), (5, 1), (6, 1), (7, 1), (8, 1), (9, 1), (10, 1), (11, 1), (12, 1), (13, 1), (14, 1), (15, 1)] [(16, 1), (17, 1), (18, 1), (19, 1), (20, 1), (21, 1), (22, 1), (23, 1), (24, 1), (25, 1), (26, 1), (27, 1), (28, 1)]


In [4]:
#from gensim.models.ldamulticore import LdaMulticore as LdaModel
from gensim.models import LdaModel
print(len(corpus))
lda = LdaModel(corpus, id2word=dictionary, num_topics=10, passes=3, alpha='auto')

34188


In [5]:
lda.print_topics()

[(0,
  '0.013*каша + 0.010*завистливый + 0.008*печь + 0.007*глиняный + 0.005*кровь + 0.005*дно + 0.005*вода + 0.005*суп + 0.005*ложка + 0.005*2.'),
 (1,
  '0.018*бог + 0.015*обжигать + 0.006*русский + 0.006*вершок + 0.004*брат + 0.004*называть + 0.004*корабль + 0.004*продавать + 0.004*первый + 0.003*адмирал'),
 (2,
  '0.007*глиняный + 0.006*государь + 0.006*сосуд + 0.006*император + 0.005*поехать + 0.005*цветочный + 0.004*выращивание + 0.004*медвежка + 0.004*нечистоты + 0.004*книга'),
 (3,
  '0.033*растение + 0.021*2 + 0.016*22 + 0.012*земля + 0.011*вода + 0.007*корень + 0.006*конь + 0.006*река + 0.006*см + 0.006*размер'),
 (4,
  '0.016*2222 + 0.013*год + 0.006*александр + 0.006*разбитый + 0.006*век + 0.005*песня + 0.004*новый + 0.004*гость + 0.004*г + 0.004*служить'),
 (5,
  '0.008*2222 + 0.008*22 + 0.006*цветок + 0.006*культура + 0.004*саша + 0.004*2 + 0.004*также + 0.004*относиться + 0.004*молча + 0.004*форма'),
 (6,
  '0.015*цветок + 0.011*цветочный + 0.011*стоять + 0.010*окно + 0.

In [6]:
import rl_wsd_labeled
senses, contexts = rl_wsd_labeled.get_contexts(rl_wsd_labeled.contexts_filename('nouns', 'RuTenTen', 'горшок'))
print(senses)
print(contexts[0])

{'3': 'Ночной горшок', '1': 'Округлый глиняный сосуд для приготовления пищи (печной горшок)', '2': 'Расширяющийся кверху сосуд с отверстием в дне (цветочный горшок)'}
(('телевизор, - ковер, , - музыкальный центр, - стол, - аквариум, - 3 шкафа, - цветы в', ' горшках', ', - мелкие аксессуары.'), '2')


In [44]:
import pymystem3
import re
MyStem = pymystem3.Mystem()

In [47]:
word_re = re.compile('\w+', re.U)
def normalize(ctx):
    left, _, right = ctx
    text = ' '.join([left, right]).strip()
    text = re.sub('\d', '2', text)
    return [w for w in MyStem.lemmatize(' '.join(word_re.findall(text)))
            if w not in stopwords and w.strip()]
print(contexts[0][0])
print(normalize(contexts[0][0]))

('телевизор, - ковер, , - музыкальный центр, - стол, - аквариум, - 3 шкафа, - цветы в', ' горшках', ', - мелкие аксессуары.')
['телевизор', 'ковер', 'музыкальный', 'центр', 'стол', 'аквариум', '2', 'шкаф', 'цветок', 'мелкий', 'аксессуар']


In [56]:
import numpy as np
documents = [dictionary.doc2bow(normalize(ctx)) for ctx, _ in contexts]
gamma, _ = lda.inference(documents)
pred_topics = gamma.argmax(axis=1)
true_labels = np.array([int(ans) for _, ans in contexts])
print(pred_topics[:10])
print(true_labels[:10])

[0 3 3 3 3 7 7 6 3 3]
[2 2 2 2 2 3 2 1 2 2]


In [55]:
from sklearn.metrics import v_measure_score, adjusted_rand_score
print('ARI:', adjusted_rand_score(true_labels, pred_topics))
print('V score:', v_measure_score(true_labels, pred_topics))

ARI: 0.17530918009027194
V score: 0.208410939616
